In [ ]:
import math
from dataclasses import dataclass

from ipywidgets import HTML
from ipywidgets import Button, VBox, HBox
from IPython.display import display

import jinja2

# Ejecuta el notebook que contine sus dependencias.
%run game.ipynb

## Clase GuiTile

Esta clase contiene información que se va a utilizar para renderizar la baldosa con la que corresponden.

El decorador @cache de las funciones crea un diccionario con los diferentes parámetros con los que se ha utilizado la función junto con los valores que se han devuelto. Antes de ejecutar la función se consulta este diccionario y en caso de que los parámetros se encuentren el él, se toma el valor diréctamente.

In [ ]:
@cache
def calculate_background_color(tile: Tile) -> str:
    """
    Calcula el color de fondo que deberá tener cada baldosa.
    :param tile: Baldosa.
    :return: Color de fondo de la baldosa.
    """
    hue = np.interp(math.log2(tile), [1, Game.rows * Game.cols], [31, 360])
    return f"hsl({hue}, 80%, 80%)"


@cache
def calculate_font_size(tile: Tile):
    """
    Calcula el tamaño de fuente del texto de la baldosa para que
    el texto no se pueda salir de la misma.
    :param tile:
    :return:
    """
    length = len(str(int(tile)))
    return min(14.9 * pow(length, -0.992), 5)


@dataclass
class GuiTile:
    value: int
    origin: Cell
    destination: Cell
    is_new: bool = False

    @property
    def background_color(self):
        return calculate_background_color(self.value)

    @property
    def font_size(self):
        return calculate_font_size(self.value)

## Mover GUI

Es igual que move() de game.ipynb, pero obtiene a su vez una lista con las baldosas que se deben renderizar.

In [ ]:
def move_gui(board: Board, direction: Direction) -> tuple[Board, list[GuiTile]]:
    """
    Obtiene un tablero resultado de mover el tablero dado en la dirección dada,
    junto con la información necesaria para mostrar los movimientos realizados.
    :param board: Tablero sobre el que realizar el movimiento.
    :param direction: Dirección en la que mover.
    :return: Tablero movido e información de movimientos.
    """
    moved = create_board()
    # Conjunto ya que facilita la consulta de pertenencia.
    merged = set()

    gui_tiles = []

    for original_cell in get_board_cells(direction):
        tile = board[original_cell]
        if not tile:
            continue

        last_cell, next_cell = get_last_and_next_cell(moved, original_cell, direction)

        if in_board(next_cell) and tile == moved[next_cell] and next_cell not in merged:
            # Combinar
            last_cell = next_cell

            gui_tiles.append(GuiTile(tile, original_cell, last_cell))
            gui_tiles.append(GuiTile(2 * tile, last_cell, last_cell, is_new=True))

            moved[last_cell] = 2 * tile
            merged.add(last_cell)
        else:
            # Mover
            moved[last_cell] = tile
            gui_tiles.append(GuiTile(tile, original_cell, last_cell))

    return moved, gui_tiles

## Añadir baldosa aleatoria GUI

Añade una baldosa aleatoria y devuelve su información para ser renderizada.

In [ ]:
def add_random_tile_gui(board: Board) -> GuiTile:
    """
    Añade una baldosa aleatoriamente y devuelve su información para ser mostrada.
    :param board: Tablero.
    :return: Información para mostrar la nueva baldosa.
    """
    cell = get_next_cell(board)
    tile = get_next_tile()
    board[cell] = tile
    return GuiTile(tile, cell, cell, is_new=True)

## Jinja2

Para realizar la creación del html correspondiente a cada estado del tablero, se ha utilizado la librería Jinja2, la cual es dependencia de Jupyter.

Esta librería permite cargar plantillas con código html junto con un lenguaje muy similar a python, el cual se ejecuta al renderizar la plantilla. A esta plantilla le podemos pasar argumentos para definir qué es lo que se debe mostrar.

In [ ]:
def get_template() -> jinja2.Template:
    """
    Carga la platilla de Jinja2.
    :return: Plantilla de Jinja2.
    """
    template_loader = jinja2.FileSystemLoader(searchpath="./")
    env = jinja2.Environment(loader=template_loader)
    return env.get_template("template.jinja")

## Clase Juego GUI

Clase que ejecuta el juego mostrando el tablero de forma gráfica como html. Esta clase no incluye controles para el jugador, por lo que será utilizada únicamente con minimax.

In [ ]:
class GameGui(Game):
    template = get_template()

    def __init__(self):
        """
        Acciones que se realizan al iniciar el juego.
        """
        self.board = create_board()

        gui_tiles = [add_random_tile_gui(self.board) for _ in range(self.initial_tiles)]

        self.html_board = HTML()
        self.refresh_html(gui_tiles)
        self.__post_init__()

    def __post_init__(self):
        display(self.html_board)

    def move(self, direction: Direction):
        """
        Realizar un movimiento.
        """
        moved, gui_tiles = move_gui(self.board, direction)
        if not np.array_equal(self.board, moved):
            self.board = moved
            gui_tiles.append(add_random_tile_gui(self.board))

        self.refresh_html(gui_tiles)

    def refresh_html(self, gui_tiles: list[GuiTile]):
        self.html_board.value = self.template.render(
            rows=Game.rows,
            cols=Game.cols,
            tiles=gui_tiles,
            has_lost=has_lost(self.board),
            has_won=has_won(self.board)
        )

## Clase juego GUI con controles

Clase juego que incluye controles para el jugador.

In [ ]:
class GameGuiPlayer(GameGui):
    def __post_init__(self):
        html = VBox(children=[
            self.html_board,
            setup_control(self),
        ])
        display(html)


def setup_control(game: GameGuiPlayer):
    """
    Añade botones para que el jugador pueda interactuar con el juego.
    :param game: Con el que se va a interactuar.
    :return: Elemento que contiene los botones.
    """
    up = Button(description="^")
    down = Button(description="v")
    right = Button(description=">")
    left = Button(description="<")

    empty = Button(description=" ")
    empty.margin = 2

    up.on_click(lambda _: game.move(Direction.UP))
    down.on_click(lambda _: game.move(Direction.DOWN))
    right.on_click(lambda _: game.move(Direction.RIGHT))
    left.on_click(lambda _: game.move(Direction.LEFT))

    return VBox([HBox([empty, up, empty]), HBox([left, down, right])])

In [ ]:
if __name__ == '__main__' and '__file__' not in globals():
    g = GameGuiPlayer()